In [2]:
from model import *
from os.path import exists

In [6]:
for p in [0.2, 0.5, 0.7]:
    for rho in [0.1, 0.2, 0.6]:
        model = Nagel_Schreckenberg_model(p, rho, 100)
        model.simulate(100)
        fname = f"base__p_{p}__rho_{rho}.gif"
        if not exists(fname):
            plot_history(model, f"base__p_{p}__rho_{rho}", f"Road simulation; p = {p}; rho = {rho}")

In [ ]:
# model = Nagel_Schreckenberg_model(0.1, 0.2, 100)
# model.simulate(2)
# plot_history(model, "base.gif", "Road simulation; p = 0.1; rho = 0.2")